<a href="https://colab.research.google.com/github/sujirou/Neural-Net-Binary-Classification/blob/main/TinyGrad_simple_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/geohot/tinygrad.git

Cloning into 'tinygrad'...
remote: Enumerating objects: 25539, done.
remote: Counting objects: 100% (2573/2573), done.
remote: Compressing objects: 100% (297/297), done.
remote: Total 25539 (delta 2407), reused 2370 (delta 2272), pack-reused 22966
Receiving objects: 100% (25539/25539), 23.66 MiB | 20.69 MiB/s, done.
Resolving deltas: 100% (18576/18576), done.


In [ ]:
cd tinygrad/

/content/tinygrad


In [ ]:
!python3 -m pip install -e .

Obtaining file:///content/tinygrad
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.2/919.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 9.5 MB/s eta 0:00:00
  Running setup.py develop for tinygrad


In [ ]:
pwd

'/content/tinygrad'

In [ ]:
import time
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import optim
import tinygrad.nn as nn
from tinygrad.helpers import flatten
from tinygrad.nn.optim import SGD
from sklearn.datasets import fetch_openml

In [ ]:
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, )
print(X.shape, y.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784) (70000,)


In [ ]:
# from tinygrad.nn import Linear

class TinyNet:
  def __init__(self):
    self.l1 = nn.Linear(784, 128, bias=False)
    self.l2 = nn.Linear(128, 10, bias=False)

  def __call__(self, x):
    x = self.l1(x)
    x = x.leakyrelu()
    x = self.l2(x)
    return x.log_softmax()

net = TinyNet()

In [ ]:
Tensor.training = True

In [ ]:
# from extra.training import sparse_categorical_crossentropy
def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

In [ ]:
# from tinygrad.nn.optim import SGD
opt = SGD([net.l1.weight, net.l2.weight], lr=3e-4)

In [ ]:
num_epochs = 2000

weight_bias_dictionary = {}
running_loss, correct, total = 0.0, 0.0, 0.0
for epoch in range(num_epochs):
    weigth_bias = {}
    start_time = time.time()

    # Randomly sample a batch
    samp = np.random.randint(0, X.shape[0], size=(64))
    batch = Tensor(X[samp].astype('float32') / 255.0, requires_grad=False)
    # Get the corresponding labels
    labels = y[samp]

    # Forward pass
    out = net(batch)

    # Compute loss
    loss = cross_entropy(out, labels)

    # Zero gradients
    opt.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    opt.step()

    # Calculate accuracy
    pred = np.argmax(out.numpy(), axis=-1)
    labels = [eval(label) for label in labels]

    acc = (pred == labels).mean()
    if epoch % 100 == 0:
      print(f"Time Taken: {time.time()-start_time:.3f}s, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.numpy():.5f}, Accuracy: {acc:.5f}")

Time Taken: 0.025s, Epoch [1/2000], Loss: 2.30219, Accuracy: 0.14062
Time Taken: 0.009s, Epoch [101/2000], Loss: 2.31252, Accuracy: 0.18750
Time Taken: 0.008s, Epoch [201/2000], Loss: 2.27642, Accuracy: 0.21875
Time Taken: 0.009s, Epoch [301/2000], Loss: 2.28463, Accuracy: 0.20312
Time Taken: 0.009s, Epoch [401/2000], Loss: 2.26676, Accuracy: 0.21875
Time Taken: 0.010s, Epoch [501/2000], Loss: 2.27412, Accuracy: 0.20312
Time Taken: 0.009s, Epoch [601/2000], Loss: 2.24246, Accuracy: 0.34375
Time Taken: 0.009s, Epoch [701/2000], Loss: 2.24648, Accuracy: 0.26562
Time Taken: 0.009s, Epoch [801/2000], Loss: 2.24841, Accuracy: 0.39062
Time Taken: 0.009s, Epoch [901/2000], Loss: 2.24378, Accuracy: 0.26562
Time Taken: 0.008s, Epoch [1001/2000], Loss: 2.24326, Accuracy: 0.31250
Time Taken: 0.008s, Epoch [1101/2000], Loss: 2.24973, Accuracy: 0.23438
Time Taken: 0.011s, Epoch [1201/2000], Loss: 2.23119, Accuracy: 0.40625
Time Taken: 0.013s, Epoch [1301/2000], Loss: 2.22418, Accuracy: 0.45312
Time

In [ ]:
# set training flag to false
Tensor.training = False

st = time.perf_counter()
avg_acc = 0
for step in range(1000):
  # random sample a batch
  samp = np.random.randint(0, X.shape[0], size=(64))
  batch = Tensor((X[samp].astype('float32') / 255.0), requires_grad=False)
  # get the corresponding labels
  labels = y[samp]

  # forward pass
  out = net(batch)

  # calculate accuracy
  pred = np.argmax(out.numpy(), axis=-1)

  labels = [eval(label) for label in labels]
  avg_acc += (pred == labels).mean()

print(f"Test Accuracy: {avg_acc / 1000}")
print(f"Time Taken To Test: {time.perf_counter() - st}")

Test Accuracy: 0.49821875
Time Taken To Test: 4.262414411000009
